In [5]:
import apriltag
import cv2
import numpy as np
import math
import mediapipe as mp

In [6]:
BoxDims = []
def CreateUI():
	UIHeight = 1080
	UIWidth = 1900

	UIBG = np.zeros((UIHeight,UIWidth,3),dtype=np.uint8)

	BoxHeight = 1000
	BoxWidth = 400
	Gap = 175

	YStart = int((UIHeight-BoxHeight)/2)

	for i in range(3):
		XStart = Gap + i*(BoxWidth+Gap)

		BoxTopLeft = (XStart,YStart)
		BoxBottomRight = (XStart+BoxWidth , YStart+BoxHeight)

		cv2.rectangle(UIBG,BoxTopLeft,BoxBottomRight,(192,242,30),-1)

		BoxDims.append((XStart,YStart,BoxWidth,BoxHeight))
	return UIBG

In [7]:
#Setup

UIBGOriginal = CreateUI()
h,w,c = UIBGOriginal.shape
options = apriltag.DetectorOptions(families="tag16h5")
detector = apriltag.Detector(options)
Angle = 0
mphands = mp.solutions.hands
mpdrawing = mp.solutions.drawing_utils
hand = mphands.Hands(min_detection_confidence = 0.7,max_num_hands = 1)

cam = cv2.VideoCapture(0,cv2.CAP_V4L2)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1900)
if not cam.isOpened():
	print("Cannot Open Camera")
	exit()

I0000 00:00:1757929995.560080    8283 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1757929995.609020   56595 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 580.65.06), renderer: NVIDIA GeForce RTX 5070 Laptop GPU/PCIe/SSE2
W0000 00:00:1757929995.625175   56572 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1757929995.636009   56574 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
while True:
	success,frame = cam.read()
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	HFrame,WFrame,_ = frame.shape

	ApriltagResults = detector.detect(gray)

	Matrix = None
	FingerOption = -1
	UIBG = UIBGOriginal.copy()
	
	if ApriltagResults:
		
		Tag = ApriltagResults[0]
		MarkerCorners = Tag.corners

		MarkerTopLeft,MarkerTopRight,MarkerBottomRight,MarkerBottomLeft = MarkerCorners
		MarkerCenter = Tag.center

		TopVector = MarkerTopRight - MarkerTopLeft
		LeftVector = MarkerBottomLeft - MarkerTopLeft

		OffsetScale = 1
		UIScale = 3

		UICenter = MarkerCenter+(LeftVector*OffsetScale)
		UIWidth = TopVector*UIScale
		UIHeight = LeftVector*UIScale
		UITopLeft = UICenter-(UIWidth/2)-(UIHeight/2)
		UITopRight = UITopLeft+UIWidth
		UIBottomLeft = UITopLeft+UIHeight
		UIBottomRight = UITopLeft+UIWidth+UIHeight

		DestinationPoints = np.array([UITopLeft,UITopRight,UIBottomRight,UIBottomLeft], dtype=np.float32)
		SourcePoints = np.array([[0,0],[w,0],[w,h],[0,h]], dtype=np.float32)
		Matrix,_ = cv2.findHomography(SourcePoints,DestinationPoints)

	RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	MpResult = hand.process(RGB)
	Pinching = False

	if MpResult.multi_hand_landmarks and Matrix is not None:
		HandLandmarks = MpResult.multi_hand_landmarks[0]
		mpdrawing.draw_landmarks(frame, HandLandmarks, mphands.HAND_CONNECTIONS)

		IndexTip = HandLandmarks.landmark[mphands.HandLandmark.INDEX_FINGER_TIP]
		ThumbTip = HandLandmarks.landmark[mphands.HandLandmark.THUMB_TIP]

		IndexPos = (int(IndexTip.x * WFrame),int(IndexTip.y * HFrame))

		try:
			InvMatrix = np.linalg.inv(Matrix)
			FingerPosOnUI = cv2.perspectiveTransform(np.array([[IndexPos]], dtype=np.float32), InvMatrix)
			fx,fy = int(FingerPosOnUI[0][0][0]),int(FingerPosOnUI[0][0][1])

			cv2.circle(UIBG, (fx, fy), 15, (0, 0, 255), -1)

			for i,(bx,by,bw,bh) in enumerate(BoxDims):
				if bx<fx<bx+bw and by<fy<by+bh:
					FingerOption = i
					break
		except np.linalg.LinAlgError:
			print("LinAlgError!")

		dist = math.hypot(ThumbTip.x - IndexTip.x , ThumbTip.y-IndexTip.y)
		IsPinching = dist < 0.05

		if FingerOption != -1 and IsPinching:
			print(f"User selected box {FingerOption+1}")
			(bx,by,bw,bh) = BoxDims[FingerOption]
			cv2.rectangle(UIBG, (bx,by), (bx+bw,by+bh), (0,255,0), -1)
			Pinching = True

	if not Pinching:
		Pinching = False

	if Matrix is not None:
		WarpedUI = cv2.warpPerspective(UIBG, Matrix, (WFrame,HFrame))

		mask = np.sum(WarpedUI, axis=2) > 0
		frame[mask] = WarpedUI[mask]

	cv2.imshow("VideoFeed",frame)

	if cv2.waitKey(1) == ord('q'):
		break

cam.release()
cv2.destroyAllWindows()

/tmp/pip-install-fhwy8tae/apriltag_637ea0c35a5445cb8841ea69e77a8f97/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(
/tmp/pip-install-fhwy8tae/apriltag_637ea0c35a5445cb8841ea69e77a8f97/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(
